In [1]:
import urllib
import pandas as pd
import ssl
import random

## Webscraper for two letter websites metadata through time using web.archive.org

In [2]:
#lista tagów
with open('html_tags_txt.txt') as f:
    tags = f.read().splitlines()

#potrzbny do bezpiecznego połączenia ze stroną WebArch
context = ssl._create_unverified_context()

#zapisanie w tabeli, w finalnej wersji będą przekazywane strony w czasie rzeczywistym
df = pd.DataFrame(0, columns=["ip_address","year","lenght","is_domein_shop"]+tags, index=range(1))

In [5]:
#utowrzenie listy stron intenetowych

alphabet = ('a','b','c')
#alphabet = ('a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','1','2','3','4','5','6','7','8','9','0')
words = [x+y+".pl" for x in alphabet for y in alphabet]

dates = [str(x)+"0701010101" for x in range(2019, 2021)]
#dates = [str(x)+"0701010101" for x in range(2005, 2021)]

websites_url_list = ["https://web.archive.org/web/"+date+"/"+word for date in dates for word in words]


In [8]:
#main
for i in range(len(websites_url_list)):
    link = websites_url_list[i]
    print(str(i)+" "+link)
    
    #załadowanie strony internetowej
    try:
        f = urllib.request.urlopen(link, context = context)           
        website_html = f.read()
        f.close()
    except:
        print("Wystąpił error w otwarciu strony")
        continue
    
    website_as_string_w = str(website_html)
    
    #jeśli nie ma "<!-- END WAYBACK TOOLBAR INSERT -->" to oznacza że strona nie została zapisana na archiwum
    if website_as_string_w.find("<!-- END WAYBACK TOOLBAR INSERT -->") < 0:
        print("Nie ma w archiwum")
        continue

    # pierwsza część strony to część renderująca WAYBACK MACHINE
    website_as_string = website_as_string_w.split("<!-- END WAYBACK TOOLBAR INSERT -->", 1)[1]
        
    #stron, które oferują domeny
    is_domein_shop = 0
    if website_as_string_w.find("domen") > 0:
        print("Domena na sprzedaż")
        is_domein_shop = 1
    
    #zliczenie tagów
    occ = []
    for line in tags:
        #print(line ,':', website_as_string.count(line))
        occ.append(website_as_string.count(line))
    
    #po __wm.wombat następują metainformacje o danej stronie internetowej w archiwum
    indx = website_as_string_w.find('__wm.wombat')
    #jeśli jej nie ma, jest wielce prawdopodobne, że strona jest niepoprawnie załadowana do archiwum
    if indx < 0:
        print("Brak __wm.wombat w pliku")
        continue
    website_info = website_as_string_w[indx + 13: indx + 200].split('","')
    ip_address = website_info[0]
    date  = website_info[1][0:4]
    lenght = len(website_as_string)

    add = [ip_address, date, lenght, is_domein_shop] + occ
    
    df.loc[i] = add
    

0 https://web.archive.org/web/20190701010101/aa.pl
1 https://web.archive.org/web/20190701010101/ab.pl
2 https://web.archive.org/web/20190701010101/ac.pl
3 https://web.archive.org/web/20190701010101/ba.pl
Domena na sprzedaż
4 https://web.archive.org/web/20190701010101/bb.pl
Nie ma w archiwum
5 https://web.archive.org/web/20190701010101/bc.pl
6 https://web.archive.org/web/20190701010101/ca.pl
Nie ma w archiwum
7 https://web.archive.org/web/20190701010101/cb.pl
Nie ma w archiwum
8 https://web.archive.org/web/20190701010101/cc.pl
Nie ma w archiwum
9 https://web.archive.org/web/20200701010101/aa.pl
10 https://web.archive.org/web/20200701010101/ab.pl
Wystąpił error w otwarciu strony
11 https://web.archive.org/web/20200701010101/ac.pl
12 https://web.archive.org/web/20200701010101/ba.pl
Domena na sprzedaż
13 https://web.archive.org/web/20200701010101/bb.pl
Nie ma w archiwum
14 https://web.archive.org/web/20200701010101/bc.pl
15 https://web.archive.org/web/20200701010101/ca.pl
Nie ma w archiwum

In [9]:
df

,ip_address,year,lenght,is_domein_shop,</a>,</abbr>,</acronym>,</address>,</applet>,</area>,...,</time>,</title>,</tr>,</track>,</tt>,</u>,</ul>,</var>,</video>,</wbr>
0,http://aa.pl:80/,2019,1156,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,http://ac.pl:80/,2019,47639,0,17,0,0,0,0,0,...,0,0,0,0,0,0,7,0,0,0
3,http://ba.pl:80/,2019,1393,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,http://www.bc.pl:80/,2019,27403,0,47,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
9,http://aa.pl/,2020,1165,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
10,https://www.ab.pl/,20200,73139,0,110,0,0,0,0,0,...,0,0,0,0,0,0,22,0,0,0
11,http://ac.pl/,2020,39036,0,75,0,0,0,0,0,...,0,0,0,0,0,0,12,0,0,0
12,http://ba.pl:80/,2020,1392,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,http://www.bc.pl/,2020,27407,0,47,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
1,https://www.ab.pl/,2019,109085,0,127,0,0,0,0,0,...,6,0,0,0,0,0,26,0,0,0
